# Inverse Distance Weighting model vs Kriging

## Table of Contents:

1. Read point data,
2. Divide dataset into two sets: modeling and validation set,
3. Perform IDW and evaluate it,
4. Perform variogram modeling on the modeling set,
5. Validate Kriging and compare Kriging and IDW validation results.

## Level: Basic

## Changelog

| Date | Change description | Author |
|------|--------------------|--------|
| 2021-05-12 | First version of tutorial | @szymon-datalions |

## Introduction

In this tutorial we will learn about the one method of validation of our Kriging model. We'll compare it to the Inverse Distance Weighting function where the unknown point value is interpolated as the weighted mean of it's neighbours. Weights are assigned by the inverted distance raised to the n-th power.

(1) **GENERAL FORM OF IDW**

$$z(u) = \frac{\sum_{i}\lambda_{i}*z_{i}}{\sum_{i}\lambda_{i}}$$,

where:

- $z(u)$: is the value at unknown location,
- $i$: is a i-th known location,
- $z_{i}$: is a value at known location $i$,
- $\lambda_{i}$: is a weight assigned to the known location $i$.

(2) **WEIGHTING PARAMETER**

$$\lambda_{i} = \frac{1}{d^{p}_{i}}$$

where:

- $d$:  is a distance from known point $z_{i}$ to the unknown point $z(u)$,
- $p$: is a hyperparameter which controls how strong is a relationship between known point and unknown point. You may set large $p$ if you want to show strong relationship between closest point and very weak influence of distant points. On the other hand, you may set small $p$ to emphasize fact that points are influencing each other with the same power irrespectively of their distance.

---

As you noticed **IDW** is a simple but powerful technique. Unfortunately it has major drawback: **we must set `p` - power - manually** and it isn't derived from the data and variogram. That's why it can be used for other tasks. Example is to use IDW as a baseline for comparison to the other techniques.

## Import packages

In [1]:
import numpy as np

import matplotlib.pyplot as plt

from pyinterpolate.idw import inverse_distance_weighting  # function for idw
from pyinterpolate.io_ops import read_point_data
from pyinterpolate.semivariance import calculate_semivariance  # experimental semivariogram
from pyinterpolate.semivariance import TheoreticalSemivariogram  # theoretical models
from pyinterpolate.kriging import Krige  # kriging models

/home/szymon/miniconda3/envs/pyintpk/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


## 1) Read point data

In [2]:
dem = read_point_data('../sample_data/point_data/poland_dem_gorzow_wielkopolski', data_type='txt')
dem

array([[15.1152409 , 52.76514556, 91.27559662],
       [15.1152409 , 52.74279035, 96.54829407],
       [15.1152409 , 52.71070647, 51.25455093],
       ...,
       [15.37034993, 52.68338343, 40.30933762],
       [15.37034993, 52.67096386, 21.94326782],
       [15.37034993, 52.64239886, 51.52513504]])

## 2) Divide dataset into two sets: modeling and validation set

In this step we will divide our dataset into two sets:

- modeling set (50%): points used for variogram modeling,
- validation set (50%): points used for prediction and results validation.

Baseline dataset will be divided randomly.

In [3]:
# Remove 40% of rows (values) to test our model

def create_model_validation_sets(dataset: np.array, frac=0.5):
    """
    Function divides base dataset into modeling and validation sets
    
    INPUT:
    :param dataset: (numpy array) array with rows of records,
    :param frac: (float) number of elements in a test set
    
    OUTPUT:
    return: modeling_set (numpy array), validation_set (numpy array)
    """

    removed_idx = np.random.randint(0, len(dem)-1, size=int(frac * len(dem)))
    validation_set = dem[removed_idx]
    modeling_set = np.delete(dem, removed_idx, 0)
    return modeling_set, validation_set

known_points, unknown_points = create_model_validation_sets(dem)

## 3) Perform IDW and evaluate it

Inverse Distance Weighting doesn't require variogram modeling or other steps. We pass power to which we want raise distance in weight denominator. Things to remember are:

- Large `power` -> closer neighbours are more important,
- `power` which is close to the **zero** -> all neighbours are important and we assume that distant process has the same effect on our variable as the closest events.

In [5]:
IDW_POWER = 2
NUMBER_OF_NEIGHBOURS = -1  # Include all points in weighting process (equation 1)

idw_predictions = []

for pt in unknown_points:
    idw_result = inverse_distance_weighting(known_points, pt[:-1], NUMBER_OF_NEIGHBOURS, IDW_POWER)
    idw_predictions.append(idw_result)

In [6]:
# Evaluation

idw_rmse = np.mean(np.sqrt((unknown_points[:, -1] - np.array(idw_predictions))**2))
print(f'Root Mean Squared Error of prediction with IDW is {idw_rmse}')

Root Mean Squared Error of prediction with IDW is 5.2961115785807005


**Clarification:** Obtained Root Mean Squared Error could serve as a baseline for further model development. To build better reference, we create four IDW models of powers:

1. 0.5,
2. 1,
3. 2,
4. 4.

In [7]:
IDW_POWERS = [0.5, 1, 2, 4]
idw_rmse = {}

for pw in IDW_POWERS:
    results = []
    for pt in unknown_points:
        idw_result = inverse_distance_weighting(known_points, pt[:-1], NUMBER_OF_NEIGHBOURS, pw)
        results.append(idw_result)
    idw_rmse[pw] = np.mean(np.sqrt((unknown_points[:, -1] - np.array(results))**2))